In [ ]:
# --- Imports and Environment Setup ---
from dotenv import load_dotenv
load_dotenv() # Load environment variables from .env file

import logging
import json # Used for loading tool configurations
import uuid # Used for generating unique thread IDs for graph execution

from pydantic import Field, BaseModel # For data validation and settings management
from typing import List, Literal # For type hinting

from langgraph.graph import MessagesState, StateGraph, START, END # Core LangGraph components for building stateful graphs
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver # For saving and resuming graph state
from langgraph.types import Command, interrupt # For controlling graph flow, e.g., human-in-the-loop

from langchain_core.prompts import PromptTemplate # For creating flexible prompts for LLMs
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage # For structuring messages in conversations
from langchain_google_genai import ChatGoogleGenerativeAI # Google Generative AI model wrapper

from final_code.utils.fetch_docs import fetch_documents # Utility function to fetch documentation (presumably for SDKs)


# --- Logging Configuration ---
# Set up the logger for application-wide logging
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs, INFO for general information
    format="%(asctime)s - %(levelname)s - %(message)s", # Log message format
    handlers=[
        # logging.FileHandler("scraper.log"),  # Option to log to a file (currently commented out)
        logging.StreamHandler()  # Log to console
    ]
)
logger = logging.getLogger(__name__) # Get a logger instance for the current module

# --- LLM Initialization ---
# Initialize the Language Model (LLM) to be used throughout the application
# Using Google's Gemini Flash model with a temperature of 0 for deterministic outputs
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0)


# --- Agent Requirement Analysis ---
# This section defines the process for understanding user requirements for building an AI agent.

# Prompt template for the Requirement Analysis LLM
template = """Your job is to get information from a user about what kind of agent they wish to build.

You should get the following information from them:

- What the objective of the agent is
- Various usecases of the agent
- Some examples of what the agent will be doing (Input and expected output pairs)

If you are not able to discern this info, ask them to clarify, you can suggest and see if user confirms the suggestions.

After you are able to discern all the information, call the tool AgentInstruction"""

class AgentInstructions(BaseModel):
    """
    Pydantic model to structure the instructions for building the Agent.
    This model is used as a tool for the LLM to output structured information.
    """
    objective: str = Field(description="What is the primary objective of the agent")
    usecases: List[str] = Field(description="What are the various responsibilities of the agent which it needs to fulfill")
    examples: str = Field(description="What are some examples of the usage of the agent (input query and expected output from the agent) ?")

class AgentBuilderState(MessagesState):
    """
    Represents the state of the main agent building graph.
    Inherits from MessagesState to include a list of messages.
    """
    agent_instructions: AgentInstructions = Field(description="The requirement analysis generated by the model.")
    # json_code: str = Field("The json code generated") # Not actively used in the provided graph logic
    python_code: str = Field(description="The Python code generated for the agent")

# class ArchitectureEvaluationState(MessagesState): # This state is not used
#     agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
#     url: str = Field("url of the agent architecture to evaluate against")


def requirement_analysis_node(state: AgentBuilderState) -> Command[Literal["requirement_analysis_node", "code_node"]]:
    """
    LangGraph node for performing requirement analysis.
    It interacts with the LLM to gather agent specifications from the user.
    If information is insufficient, it interrupts the graph for user input.
    Otherwise, it proceeds to the code generation node.
    """
    logger.info("Executing requirement_analysis_node")
    llm_with_tool = llm.bind_tools([AgentInstructions]) # Bind the AgentInstructions Pydantic model as a tool
    
    # Invoke the LLM with the system prompt and current message history
    response = llm_with_tool.invoke([SystemMessage(content=template)] + state["messages"])
    
    if not response.tool_calls: # If no tool call, means LLM needs more info or is just conversing
        logger.info("LLM requires more information or is in conversation.")
        # Interrupt the graph to get user input. The content of 'value' will be sent to the user.
        value = interrupt(response.content)
        # Return a command to stay in the current node and update messages with LLM's response and the interruption
        return Command(goto="requirement_analysis_node", update={"messages": [response, HumanMessage(content=value)]})
        
    # If a tool call is present, it means the LLM has gathered the required information
    logger.info("LLM successfully called AgentInstructions tool.")
    agent_instructions_args = response.tool_calls[0]["args"]
    agent_instructions = AgentInstructions(**agent_instructions_args)
    
    # Return a command to proceed to 'code_node' and update state
    return Command(
        goto="code_node",
        update={"messages": [response], "agent_instructions": agent_instructions}
    )

# --- Agent Code Generation ---
# This section focuses on generating the main Python code for the agent based on the gathered requirements.

CODE_GEN_PROMPT = PromptTemplate.from_template("""
You are an expert Python programmer specializing in AI agent development via the Langgraph and Langchain SDK . Your primary task is to generate compilable, logical, and complete Python code for a LangGraph state graph based on user input below. You must prioritize LLM-based implementations for relevant tasks and consider advanced graph architectures.

**Input:**
<INPUT>
<OBJECTIVE>
{objective}
</OBJECTIVE>
<USECASES>
{usecases}
</USECASES>
<EXAMPLES>
{examples}
</EXAMPLES>
</INPUT>
---
**Phase 1: Evaluating best architecture for the given INPUT**

1.  **Identify Potential Architectures:** Consider if the described INPUT aligns with or would benefit from known advanced LangGraph architectures such as:
    * **Plan and Execute**: Does the INPUT imply an agent which might need a planning step (e.g., breaking down a complex task) followed by the execution of those plans by one or more action nodes?
    * **Agent Supervisor / Hierarchical Agent Teams**: Is the INPUT best served by a supervisor agent dispatching tasks to specialized worker agents, or a hierarchy of agents making decisions and delegating?
    * **Multi-Agent Collaboration (e.g., Swarm Architecture)**: Does the problem benefit from multiple agents working in parallel or collaboratively, perhaps sharing insights or contributing to a common goal?
    * **Reflection / Self-Correction (e.g., Self-Discover frameworks)**: Are there indications of iterative refinement, where results are evaluated and the process is adjusted?
    * **Human in the Loop (HITL)**: Does the `description` of any node, or the overall process, imply a need for human review, approval, correction, or explicit input at specific stages (e.g., before executing a critical action, when confidence is low, or for subjective assessments)?

2.  **Architectural Decision:**
    * If you determine that one or more of these architectures are strongly applicable to the INPUT, choose to implement it.
    * If no specific advanced architecture seems directly applicable for the given INPUT, proceed with a standard stateful graph construction based on the explicit langgraph nodes and edges.

3.  **Initial Comment:** At the very beginning of your generated Python script, include a comment block stating:
    * Which LangGraph architecture(s) (if any) you've identified and chosen to implement, with a brief justification based on your interpretation of the INPUT, provide dry runs of the usecases/examples.
    * If you are proceeding with a standard graph, mention that.

4. Generate a JSON representation of the architecture you have chosen, including:
a.  `nodes`: A dictionary where each key is a unique node ID. The value for each node ID is an object containing:
    * `id`: The node's identifier.
    * `schema_info`: A string describing the structure of the `GraphState` (e.g., "GraphState:\\n type: TypedDict\\n fields:\\n - name: input\\n type: str..."). You will need to parse this to define the `GraphState` TypedDict.
    * `input_schema`: The expected input schema for the node (typically "GraphState").
    * `output_schema`: The schema of the output produced by the node (typically "GraphState", indicating a partial update).
    * `description`: A natural language description of what the node does. This is crucial for determining implementation strategy and overall architecture.
    * `function_name`: The suggested Python function name for this node.
    * `code` (optional): A string containing Python code for the node's function. **Treat this `code` primarily as an illustration or a very basic version. Prioritize LLM-based solutions if the `description` suggests a more robust approach is needed.**

b.  `edges`: A list of objects, each describing a directed edge in the graph. Each edge object contains:
    * `source`: The ID of the source node (or "__START__" for the graph's entry point).
    * `target`: The ID of the target node (or "__END__" for a graph termination point).
    * `routing_conditions`: A natural language description of the condition under which this edge is taken, especially for conditional edges.
    * `conditional`: A boolean flag, `true` if the edge is part of a conditional branch, `false` otherwise.


---

** Phase 2: Graph Creation
**Phase 2: Python Code Generation**

Generate a single, self-contained, and compilable Python script that implements your chosen strategy.

1.  **Imports:** Include all necessary Python libraries (e.g., `typing`, `langgraph.graph`, `langgraph.checkpoint.memory`, LLM client libraries like `langchain_openai`, `langchain_google_genai`, `langchain_core.pydantic_v1`, `langchain_core.tools`, `re`).

2.  **State Definition (`GraphState`):**
    * Define a `GraphState` class using `MessagesState` (langgraph prebuilt class).

3.  **Node Implementation (Python Functions):**
    For each conceptual node in your chosen architecture (these may map directly to JSON you define):
    * Create a Python function. This function must accept the `GraphState` and return a dictionary representing the partial update to the state.
    * **Decision Logic for Implementation (Prioritize LLM, No Mock Data):**
        * **Default to LLM-Based Solutions:** Your default stance should be to implement an **LLM-based solution** if the node's `description` (from JSON or your architectural design) suggests tasks like:
            * Natural Language Understanding (NLU)
            * Complex classification or routing
            * Content generation or summarization
            * Tool selection and usage
            * Planning or complex decision-making.
            * Any task where an LLM would provide more robust, flexible, or intelligent behavior than simple hardcoded logic.
        * **Handling Provided `code`:** If `code` is present in the JSON for a node, treat it as a **low-priority hint or a simplistic example**. Do **not** simply copy it if an LLM approach is more appropriate for the described task.
        * **Algorithmic Logic (Use Sparingly):** Only use purely algorithmic Python code (like from the `code` attribute or written new) if the node's task is genuinely simple, deterministic (e.g., basic data formatting, fixed calculation), *and* an LLM would offer no significant benefit for that specific, narrow function.
        * **Functional LLM Calls:** When an LLM is used, instantiate a generic model (e.g., `llm = ChatOpenAI(model="gpt-3.5-turbo")` or `llm = ChatGoogleGenerativeAI(model="gemini-pro")`) and include a **functional, descriptive prompt** relevant to the node's task. Ensure the code for the LLM call is complete and not just a comment. Add a `TODO` comment for the user to specify API keys and potentially refine the model/prompt.
        * **No Mock Data:** Generated functions must be logical and aim for completeness. **Avoid using mock data or overly simplistic placeholder logic** where an LLM or a proper algorithmic implementation is expected.
        * **Structured Output & Tools:** If the task implies structured output from an LLM or the use of tools, define necessary Pydantic models and/or LangChain tools, and integrate them with the LLM call.
            * Define a Pydantic model (e.g., `from langchain_core.pydantic_v1 import BaseModel, Field`) representing the desired structured output.
            * If implementing an LLM call, configure it to use the Pydantic model for its output (e.g., with OpenAI's function calling/tool usage features, or by instructing the LLM to generate JSON conforming to the model).
        * **Tool Definition and Usage:** If a node's `description` (or your architectural design) implies the LLM within that node needs to interact with external systems, perform specific actions, or fetch data (e.g., "search customer database," "get weather update"):
                * Define these capabilities as discrete LangChain tools using the `@tool` decorator (e.g., `from langchain_core.tools import tool`).
                * **Crucially, each tool's internal Python function should be self-contained and directly perform its advertised action** (e.g., make a specific API call to an external service, run a local script, perform a calculation, retrieve data algorithmically). **Avoid embedding a *new, separate general-purpose LLM call within the tool's own implementation logic* unless the tool's explicit and documented purpose is to be a specialized, self-contained sub-agent (which is an advanced case).** The primary LLM within the graph node is responsible for *deciding to call* the tool and for interpreting its output.
                * Bind these well-defined tools to the LLM instance operating within that graph node. The node's LLM will then intelligently decide when to call a tool and with what inputs.
        * **Human in the Loop Nodes:** If you've designed a HITL step as a dedicated node, its function might primarily format data for human review and then process the subsequent human input (which would be added to the state, potentially by an external mechanism or a subsequent node). The graph might pause using an interruption mechanism tied to this node.
        * **State Coherence:** Ensure variable assignments and updates within node functions are coherent with the `GraphState` definition and how state is managed in LangGraph.

4.  **Graph Construction (`StatefulGraph`):**
    * Instantiate `StatefulGraph(GraphState)`.
    * Add each implemented node function to the graph using `graph.add_node("node_id", node_function)`.
    * Set the graph's entry point using `graph.add_edge(START, "entry_node_id")` where `"entry_node_id"` is the target of the edge originating from `"__START__"`.

5.  **Edge Implementation:**
    * Iterate through the `edges` list in the JSON.
    * **Regular Edges:** If `conditional` is `false`:
        * If `target` is `__END__`, use `graph.add_edge(source_node_id, END)`.
        * Otherwise, use `graph.add_edge(source_node_id, target_node_id)`.
    * **Conditional Edges:** If `conditional` is `true`:
        * The `source` node of these conditional edges is expected to produce some output in the `GraphState` (e.g., an `intent` field) that determines the next path.
        * Create a separate routing function (e.g., `def route_after_source_node(state: GraphState) -> str:`).
        * This routing function must inspect the relevant fields in the `state` and return the string ID of the next node to execute, based on the logic described in the `routing_conditions` for each conditional edge originating from that source.
        * Use `graph.add_conditional_edges(source_node_id, routing_function, {{ "target_id_1": "target_id_1", "target_id_2": "target_id_2", ... "__END__": END }})`. The keys in the dictionary are the possible return values from your routing function, and the values are the actual node IDs or `END`.

6.  **Compilation:**
    * Instantiate an `InMemoryCheckpointer`: `checkpointer = InMemoryCheckpointer()`.
    * Compile the graph: `final_app = graph.compile(checkpointer=checkpointer)`. The compiled graph must be assigned to a variable named `final_app`.

---
**Phase 3: Required Keys/Credentials Identification**

After generating the complete Python script, add a separate section at the end of your response, clearly titled:
`## Required Keys and Credentials`

In this section, list all environment variables or API keys a user would need to set for the generated code to execute successfully (e.g., `OPENAI_API_KEY`, `GOOGLE_API_KEY`, tool-specific keys). If no external keys are needed, state that.

---
**Important Considerations (General):**
* The primary goal is **compilable, logical, and functionally plausible Python code** that intelligently interprets the JSON input.
* Focus on creating a system that leverages LLMs effectively for tasks suited to them.
* Ensure node functions correctly update and return relevant parts of the `GraphState`.
* If the provided `code` in the JSON uses specific libraries (e.g., `re`), make sure the corresponding import is included at the top of the script.
* Handle `__START__` and `__END__` correctly in edge definitions. `langgraph.graph.START` and `langgraph.graph.END` should be used.

Please generate the Python code and the list of required keys now:
""")

def code_node(state: AgentBuilderState):
    """
    LangGraph node to generate the final Python code for the agent.
    It uses the gathered agent_instructions and the CODE_GEN_PROMPT.
    """
    logger.info("Executing code_node")
    instructions: AgentInstructions = state["agent_instructions"]
    
    # Invoke LLM to generate code based on the detailed prompt and instructions
    code_output = llm.invoke([HumanMessage(content=CODE_GEN_PROMPT.format(
        objective=instructions.objective,
        usecases=instructions.usecases,
        examples=instructions.examples
    ))])
    
    logger.info("Python code generated by LLM.")
    # Return the generated Python code and an AI message
    return {
        "messages": [AIMessage(content="Generated final python code!")],
        "python_code": code_output.content,
    }

# --- Tool/Function Definition Sub-Graph Components ---
# This section defines components for a sub-graph that creates individual Python functions (tools).

# Load tool/SDK information from JSON files.
# Note: Hardcoded paths might need adjustment depending on the deployment environment.
try:
    with open("D:\\AgentAgent\\AgentAgent\\experiments\\tool_creation\\tools_link_json.json") as f:
        dict_tool_link = json.load(f)
    with open("D:\\AgentAgent\\AgentAgent\\experiments\\tool_creation\\tools_doc_json.json") as f:
        dict_tool_doc = json.load(f)
except FileNotFoundError:
    logger.error("Tool link/doc JSON files not found. Please check the paths.")
    dict_tool_link = {} # Default to empty dict if file not found
    dict_tool_doc = {}  # Default to empty dict if file not found


def lowercase_keys(input_dict: dict) -> dict:
    """
    Utility function to convert all keys in a dictionary to lowercase.
    """
    return {k.lower(): v for k, v in input_dict.items()}

# Normalize keys in the loaded tool dictionaries
dict_tool_link = lowercase_keys(dict_tool_link)
dict_tool_doc = lowercase_keys(dict_tool_doc)

# Prompt for initial analysis of a function's description
Initial_prompt = """You are an expert python developer. You will be given a description of a python function. 

Your job is to estimate and extract the following information:

- What exactly does this python do. What is the detailed objective of the function. Please write 1-5 lines
- Suggest or extract the name of the the function
- What would be the inputs/arguements required into this function to make it work. Please all mentioned the type of each input
- WHat would be output produced by this input. Please mention the output type 

Here is the description of the function you need to create:
<description>
{desc}
</description>
"""

class FunctionInstructions(BaseModel):
    """
    Pydantic model to structure the definition of a Python function (tool).
    Used for structured output from the LLM during function analysis and code generation.
    """
    objective: str = Field(description="what does this python function do")
    name: str = Field(description="name of the python function")
    input: List[str] = Field(description="what would be the input arguements to this function along with the types")
    output: List[str] = Field(description="what would be the output/return attributes for the function along with the types")
    name_toolkit: str = Field(description="what would be the toolkit/ code SDK that will be used") # Name of the SDK
    code: str = Field(description="the final python code for the function")

# class CodebuilderState(BaseModel): # This state is not used
#     """Instructions for defining a python function"""
#     code: str = Field(description= "tailored code for the python function")


def functional_analysis_node(state: FunctionInstructions):
    """
    LangGraph node for analyzing a function description.
    It uses the LLM with structured output (FunctionInstructions) to parse the description.
    """
    logger.info(f"Executing functional_analysis_node for: {state.objective[:50]}...") # Log snippet of objective
    llm_with_structured_output = llm.with_structured_output(FunctionInstructions)
    
    # Invoke LLM to get structured information about the function
    functionalReport: FunctionInstructions = llm_with_structured_output.invoke(
        [HumanMessage(content=Initial_prompt.format(desc=state.objective))]
    )
    logger.info(f"Functional analysis complete for {functionalReport.name}.")
    return {
        "messages": [AIMessage(content="Generated JSON code for function analysis!")], # Consider a more descriptive message
        "objective": functionalReport.objective,
        "name": functionalReport.name,
        "input": functionalReport.input,
        "output": functionalReport.output,
        # name_toolkit and code are not set here, they are determined in subsequent nodes
    }

# Prompt for writing the code of a function using a specific SDK
write_code_prompt = """
You are a skilled code generation assistant. Your task is to create executable code using the following information:
- SDK Documentation: The provided documentation outlines the functionalities and usage details of the SDK. Use this as the reference for constructing your code.
- Objective: A clear description of what the code is intended to achieve.
- Input: The expected input for the code (e.g., variables, parameters, data types).
- Output: The desired result or outcome of the code (e.g., format, type, or structure).
- SDK Name: The name of the SDK that must be used in the code.

Your goal is to generate executable code that:
- Adheres to the requirements outlined above.
- Follows standard coding practices and is optimized for readability and efficiency.
- Utilizes the specified SDK appropriately based on the documentation provided.
- Only return a self contained function
- Your output should only contain a code block containing the required function and nothing else. Please do no include any explainantions
- Write your code in python
- Please also provide which API keys will be required and define the API keys as part of the function
- Please also write the doc string for the python function
- Ensure that the function you produce is decorated with @tool. That means its defination should be preceeded by '@tool' in the line above

Here are some details about the python function you will be creating:
<objective>
{objective}
</objective>

<input schema>
{inputs}
</input schema>

<output schema>
{output}
</output schema>

<name of function>
{name}
</name of function>

Documentation for SDK that might be helpful:
<documentation>
{docs}
</documentation>
"""

# Prompt for selecting the best SDK for a given function
Best_sdk_prompt = """
You are a highly specialized language model designed to assist in selecting the most suitable SDK for a given use case. You are provided with the following:
- A dictionary containing pairs of SDK names and their respective descriptions.
- Requirements for a piece of code, including the objective, input, and output.

Your task is to:
- Identify the SDK from the provided dictionary whose description best matches the given use case described in the code requirements.
- Also give preferences to SDKs that are generally more well known or are used more frequently in the industry (Use google tools for anything search related)
- Return only the name of the matching SDK without any additional text or formatting.
- Please ensure that the string you return is a valid key of the dictionary you get as input. PLEASE VERIFY THAT THE STRING YOU RETURN EXISTS AS A KEY IN THE INPUT DICTIONARY

Input Example:
Dictionary:
{{
"SDK_A": "[SDK_CC_ABC]Provides tools for web scraping and data extraction.",
"SDK_B": "Enables natural language processing for unstructured text.",
"SDK_C": "Facilitates the integration of payment gateways in applications."
}}
Code Requirements:
Objective: Extract data from multiple web pages.
Input: URLs of the web pages.
Output: Structured data in JSON format.

Expected Output:
SDK_A


Input :
<dictionary>
{dictionary}
</dictionary>

<objective>
{objective}
</objective>

<input schema>
{inputs}
</input schema>

<output schema>
{output}
</output schema>

<name of function>
{name}
</name of function>
"""

def sdk_production_node(state: FunctionInstructions):
    """
    LangGraph node to identify the most suitable SDK for the function.
    It uses the LLM with the Best_sdk_prompt and the loaded SDK documentation.
    """
    logger.info(f"Executing sdk_production_node for function: {state.name}")
    objective_agent: str = state.objective
    name: str = state.name
    input_args: List[str] = state.input
    output_args: List[str] = state.output
    
    response = llm.invoke([HumanMessage(content=Best_sdk_prompt.format(
        objective=objective_agent,
        inputs=input_args,
        output=output_args,
        name=name,
        dictionary=dict_tool_doc # Using the dictionary of SDK descriptions
    ))])
    
    sdk_name = response.content.lower().strip()
    logger.info(f"SDK identified: {sdk_name} for function {name}")
    
    # Ensure the identified SDK is valid
    if sdk_name not in dict_tool_doc:
        logger.warning(f"LLM returned an SDK name ('{sdk_name}') not found in dict_tool_doc. Defaulting or error handling might be needed.")
        # Potentially raise an error or pick a default if this happens
    
    return {
        "messages": [AIMessage(content=f"SDK '{sdk_name}' identified for function {name}.")],
        "name_toolkit": sdk_name
    }

def code_production_node(state: FunctionInstructions):
    """
    LangGraph node to generate the Python code for the function using the identified SDK.
    It fetches SDK documentation and uses the write_code_prompt.
    """
    logger.info(f"Executing code_production_node for function: {state.name} using SDK: {state.name_toolkit}")
    objective_agent: str = state.objective
    name: str = state.name
    input_args: List[str] = state.input
    output_args: List[str] = state.output
    toolkit: str = state.name_toolkit
    
    docs = ""
    if toolkit in dict_tool_link:
        docs = fetch_documents(dict_tool_link[toolkit]) # Fetch documentation for the chosen SDK
        logger.info(f"Fetched documentation for SDK: {toolkit}")
    else:
        logger.warning(f"No documentation link found for SDK: {toolkit}. Proceeding without SDK-specific docs.")

    response = llm.invoke([HumanMessage(content=write_code_prompt.format(
        objective=objective_agent,
        inputs=input_args,
        output=output_args,
        name=name,
        docs=docs,
    ))])
    
    logger.info(f"Code generated for function: {name}")
    return {
        "messages": [AIMessage(content=f"Generated code for tool/function: {name}")],
        "code": response.content
    }

# --- Tool Information Graph Definition (`tool_infograph`) ---
# This graph orchestrates the creation of a single tool/function.
# It takes a function description and produces its Python code.

# Using InMemorySaver for this sub-graph as its state might not need to persist long-term
tool_info_checkpointer = InMemorySaver()
tool_info_workflow = StateGraph(FunctionInstructions) # Define state type for this graph

# Add nodes to the tool information graph
tool_info_workflow.add_node("func_analysis", functional_analysis_node)
tool_info_workflow.add_node("sdk_write", sdk_production_node)
tool_info_workflow.add_node("code_write", code_production_node)

# Define edges for the tool information graph
tool_info_workflow.add_edge(START, "func_analysis") # Start with functional analysis
tool_info_workflow.add_edge("func_analysis", "sdk_write") # Then identify SDK
tool_info_workflow.add_edge("sdk_write", "code_write")   # Then write code
tool_info_workflow.add_edge("code_write", END)          # End after code writing

# Compile the tool information graph
tool_infograph = tool_info_workflow.compile(checkpointer=tool_info_checkpointer)
logger.info("Tool information graph (tool_infograph) compiled.")


# --- Tool Compilation Sub-Graph Components ---
# This section defines components for a sub-graph that identifies tools in the main agent code,
# generates their implementations (using tool_infograph), and compiles them back.

class ToolCollectorState(MessagesState): # Renamed from 'toolcollector' for convention
    """
    State for the graph that collects and compiles multiple tool codes.
    """
    total_code: List[str] = Field(default_factory=list, description="List of generated Python code snippets for tools.")
    compiled_code: str = Field(description="The main agent Python code, potentially with placeholders for tools.")

# test_message = """ ... """ # This variable was not used, so it's removed.

# Prompt to extract descriptions of functions decorated with @tool
tool_desc_prompt = """
You are an AI assistant designed to analyze Python code. Your task is to identify all function definitions in the provided Python snippet that are decorated with @tool. You must return a dictionary where:
- The keys are the names of the identified functions.
- You only need to pick up a function if it is decorated with '@tool' or '@tool' just preceeds the function. Otherwise leave the function alone
- The values are descriptions of what each function is supposed to do. If a function contains a docstring, extract it as the description. If a docstring is missing, infer the function's purpose from its structure and comments.
- The output should just be a json. it should not include "```json" and "```" at the start or end of it. it should start with a "{{" and end with a "}}"
Example Input:
@tool
def calculate_area(length, width):
    "Calculates the area of a rectangle."
    return length * width

@tool
def greet(name):
    return f"Hello, {{name}}!"


Expected Output:
{{
    "calculate_area": "Calculates the area of a rectangle.",
    "greet": "Greets a user by name."
}}


Instructions:
- Identify functions that have the @tool decorator.
- Extract function names and descriptions (either from docstrings or inferred).
- Return the output as a structured JSON.
- Please only return a json object that can be converted into a json directly. DO NOT RETURN ANYTHING OTHER THAN A JSON. it should start with a "{{" and end with a "}}" and there should not be any markers in the response to show that it is a json

Python code:
<code>
{code}
</code>
"""

# Prompt to compile main code with generated tool function definitions
tool_compile_prompt = """
You are python code writing expert. You are given 2 snippets of code, your job is to combine them. 
The first snippet of code contains a compilable code with some functions compilable but empty. 
The second snippet of code contains the defination of those functions. 
Please fo through the second snippet of code, match the function in the first snippet and replace the functional definition written in the first snippet with one found in second snippet

Please only return compilable python code
Here are the code snippets:
<code_snippet1>
{complete_code}
</code_snippet1>
<code_snippet2>
{functions}
</code_snippet2>
"""

def graph_map_step(state: ToolCollectorState):
    """
    LangGraph node to identify tools in the compiled agent code and generate their implementations.
    It iterates over functions marked with @tool, invokes `tool_infograph` for each,
    and collects the generated code.
    """
    logger.info("Executing graph_map_step to identify and generate tool implementations.")
    current_code = state['compiled_code']
    
    # Use LLM to find @tool decorated functions and their descriptions
    response_1 = llm.invoke([HumanMessage(content=tool_desc_prompt.format(code=current_code))])
    response_content = response_1.content.strip()
    
    json_objects = {}
    try:
        # Handle potential "```json\n" prefix and "\n```" suffix from LLM
        if response_content.startswith("```json"):
            response_content = response_content[7:]
        if response_content.endswith("```"):
            response_content = response_content[:-3]
        json_objects = json.loads(response_content.strip())
        logger.info(f"Identified tools for implementation: {list(json_objects.keys())}")
    except json.JSONDecodeError as e:
        logger.error(f"Failed to parse JSON from tool_desc_prompt output: {e}")
        logger.error(f"LLM response content was: {response_content}")
        # If parsing fails, no tools will be generated in this step.
        # Consider how to handle this error robustly (e.g., retry, skip, or error state).

    generated_tool_codes = []
    for tool_name, tool_description in json_objects.items():
        logger.info(f"Generating implementation for tool: {tool_name} - Description: {tool_description[:50]}...")
        # Each tool generation runs in its own "thread" or configuration for the sub-graph
        thread_id = str(uuid.uuid4())
        config = {"configurable": {"thread_id": thread_id}}
        
        # Initial state for the tool_infograph
        initial_tool_state = {
            "objective": tool_description, # The description from tool_desc_prompt becomes the objective
            "name": tool_name,
            "input": [], # Inputs/outputs could be further refined or extracted by tool_desc_prompt
            "output": [],
            "name_toolkit": "", # To be determined by sdk_production_node in tool_infograph
            "code": ""          # To be generated by code_production_node in tool_infograph
        }

        # Stream updates from the tool_infograph execution (optional, good for logging/debugging)
        for output_update in tool_infograph.stream(initial_tool_state, config, stream_mode="updates"):
            logger.debug(f"Update from tool_infograph for {tool_name}: {output_update}")
        
        # Get the final state of the tool_infograph to retrieve the generated code
        final_tool_state = tool_infograph.get_state(config)
        if "code" in final_tool_state.values and final_tool_state.values["code"]:
            generated_tool_codes.append(final_tool_state.values["code"])
            logger.info(f"Successfully generated code for tool: {tool_name}")
        else:
            logger.warning(f"No code generated for tool: {tool_name}. State: {final_tool_state.values}")
            
    return {
        "messages": [AIMessage(content="Tool identification and individual code generation complete.")],
        "total_code": generated_tool_codes, # List of code strings for each tool
        "compiled_code": current_code # Pass along the original compiled code
    }

def compile_tool_code_node(state: ToolCollectorState): # Renamed for clarity
    """
    LangGraph node to combine the main agent code with the generated tool function codes.
    Uses an LLM with `tool_compile_prompt` to perform the merge.
    """
    logger.info("Executing compile_tool_code_node to merge tool codes with main agent code.")
    tool_code_list = state['total_code']
    main_agent_code = state['compiled_code'] # Renamed for clarity
    
    if not tool_code_list:
        logger.info("No tool codes to compile. Returning original agent code.")
        return {
            "messages": [AIMessage(content="No new tool functions to compile.")],
            "compiled_code": main_agent_code # Return original if no tools were generated
        }

    full_tool_code = "\n\n".join(tool_code_list) # Join tool codes with newlines
    
    # Use LLM to merge the main agent code with the generated tool definitions
    response = llm.invoke([HumanMessage(content=tool_compile_prompt.format(
        complete_code=main_agent_code,
        functions=full_tool_code
    ))])
    
    logger.info("Main agent code compiled with tool function definitions.")
    # The response from this LLM call is expected to be the final, complete Python code
    return {
        "messages": [AIMessage(content=response.content)], # Storing the LLM's final code as a message for now
        "compiled_code": response.content # Update compiled_code with the final merged code
    }

# --- Tool Compilation Graph Definition (`tool_compile_graph`) ---
# This graph manages the process of finding @tool placeholders in the main generated code,
# generating their implementations, and then merging them back.
tool_compile_workflow = StateGraph(ToolCollectorState)

# Add nodes to the tool compilation graph
tool_compile_workflow.add_node("graph_map_step", graph_map_step)
tool_compile_workflow.add_node("compile_tools", compile_tool_code_node) # Renamed node

# Define edges for the tool compilation graph
tool_compile_workflow.add_edge(START, "graph_map_step")
tool_compile_workflow.add_edge("graph_map_step", "compile_tools")
tool_compile_workflow.add_edge("compile_tools", END)

# Compile the tool compilation graph
# This graph doesn't seem to require a checkpointer in this setup,
# but adding one if state needs to be inspected or persisted.
tool_compile_graph = tool_compile_workflow.compile()
logger.info("Tool compilation graph (tool_compile_graph) compiled.")


# --- Main Agent Generation Graph ---
# This is the primary graph that orchestrates the entire agent building process.

def call_tool_subgraph_node(state: AgentBuilderState): # Renamed for clarity
    """
    LangGraph node that invokes the `tool_compile_graph` sub-graph.
    It passes the currently generated agent code (which might have tool placeholders)
    to the sub-graph for tool implementation and compilation.
    """
    logger.info("Executing call_tool_subgraph_node.")
    # Initial state for the tool_compile_graph
    subgraph_input = {
        "compiled_code": state["python_code"],
        # total_code will be populated by graph_map_step within the subgraph
        "messages": [] # Start with fresh messages for the subgraph
    }
    
    # Invoke the tool compilation sub-graph
    subgraph_output = tool_compile_graph.invoke(subgraph_input)
    
    logger.info("Tool compilation sub-graph finished.")
    # Update the main graph's python_code with the output from the sub-graph
    return {"python_code": subgraph_output["compiled_code"], "messages": subgraph_output["messages"]}


# Checkpointer for the main agent generation graph
main_checkpointer = MemorySaver()
main_workflow = StateGraph(AgentBuilderState) # Define state type

# Add nodes to the main workflow
main_workflow.add_node("requirement_analysis_node", requirement_analysis_node)
main_workflow.add_node("code_node", code_node)
main_workflow.add_node("tool_subgraph_processing", call_tool_subgraph_node) # Renamed node

# Define edges for the main workflow
main_workflow.add_edge(START, "requirement_analysis_node")             # Start with requirement analysis
main_workflow.add_edge("requirement_analysis_node", "code_node")        # Then generate initial code
main_workflow.add_edge("code_node", "tool_subgraph_processing")    # Then process/implement tools via sub-graph
main_workflow.add_edge("tool_subgraph_processing", END)            # End after tool processing

# Compile the main agent generation graph
agent_generator_graph = main_workflow.compile(checkpointer=main_checkpointer)
logger.info("Main agent generator graph compiled.")


# --- Example Invocation (Optional) ---
# This section can be used to demonstrate how to run the main graph.
# For example:
# if __name__ == "__main__":
#     logger.info("Starting agent generation process...")
#     initial_input = {"messages": [HumanMessage(content="I want to build an agent that can tell me the weather.")]}
#     config = {"configurable": {"thread_id": "user-thread-1"}}
    
#     for event in agent_generator_graph.stream(initial_input, config=config):
#         for key, value in event.items():
#             logger.info(f"Event from graph: {key} - {value}")
#             if key == "requirement_analysis" and isinstance(value, dict) and 'messages' in value:
#                 last_message = value['messages'][-1]
#                 if isinstance(last_message, HumanMessage) and last_message.content.startswith("__interrupt__"):
#                     # This is a simplified way to handle interruption for demo.
#                     # In a real app, you'd present this to the user and get their input.
#                     user_response = input(f"Agent asks: {last_message.content[len('__interrupt__'):].strip()} Your response: ")
#                     # How to reinvoke or continue with new input needs careful handling with stream/invoke
#                     # For simplicity, this example doesn't fully implement the interactive loop here.
#                     logger.info(f"User input received: {user_response} (manual continuation needed for stream)")


#     final_state = agent_generator_graph.get_state(config)
#     logger.info("\n--- Final Generated Python Code ---")
#     print(final_state.values.get("python_code"))
#     logger.info("Agent generation process complete.")

2025-06-03 02:34:12,627 - INFO - Tool information graph (tool_infograph) compiled.
2025-06-03 02:34:12,636 - INFO - Tool compilation graph (tool_compile_graph) compiled.
2025-06-03 02:34:12,642 - INFO - Main agent generator graph compiled.


In [6]:
query= " need to create a worklow with the objective of managing my social media.  It should be able to tell me trends from social media for sports, get me all the relevant people I should contant for a spsonsoring a specific post,  suggest me content I should be posting, make content for me if I give it a description * Identifying social media trends in sports. *   Finding relevant people for sponsoring specific posts. *   Suggesting content to post. *   Generating content based on a description.  Examples: Given I ask it about trends in football, it goes through recent viral reels in football and tell me Q&A reels are trending If I tell it I want people who would be interested in a post about UCL football, it gives me current players like Dembele etc If I tell it I want to make a post about UCL football, it goes through what is trending and an agent that thinks about social media posts and tells me we should post a highlight reel"

In [7]:
# Run the graph until the interrupt is hit.
config = {"configurable": {"thread_id": "some_id"}}
result = agent_generator_graph.invoke({"messages": HumanMessage(content=query)}, config=config) 




2025-06-03 02:34:15,765 - INFO - Executing requirement_analysis_node
2025-06-03 02:34:18,900 - INFO - LLM successfully called AgentInstructions tool.
2025-06-03 02:34:18,903 - INFO - Executing code_node
2025-06-03 02:34:56,649 - INFO - Python code generated by LLM.
2025-06-03 02:34:56,652 - INFO - Executing call_tool_subgraph_node.
2025-06-03 02:34:56,657 - INFO - Executing graph_map_step to identify and generate tool implementations.
2025-06-03 02:35:00,351 - INFO - Identified tools for implementation: ['search_viral_content', 'influencer_search']
2025-06-03 02:35:00,351 - INFO - Generating implementation for tool: search_viral_content - Description: Searches for recent viral social media content or ...
2025-06-03 02:35:00,355 - INFO - Executing functional_analysis_node for: Searches for recent viral social media content or ...
2025-06-03 02:35:04,252 - INFO - Functional analysis complete for search_trending_content.
2025-06-03 02:35:04,254 - INFO - Executing sdk_production_node for f

### Langsmith link: https://smith.langchain.com/public/b8c5cb7a-7c20-4f72-bbc0-300becdcd4ab/r

In [ ]:
import operator
import os
import json
from typing import Annotated, List, Literal, TypedDict

from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemoryCheckpointer
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import MessagesState

# New imports for the actual tool implementations
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_community.utilities.golden_query import GoldenQueryAPIWrapper

# --- Phase 1: Architectural Decision and Justification ---
"""
LangGraph Architecture: Agent Supervisor / Hierarchical Agent Teams

Justification:
The problem of "managing social media" naturally breaks down into distinct, specialized sub-tasks:
1.  Identifying social media trends.
2.  Finding relevant people for sponsoring posts.
3.  Suggesting content ideas.
4.  Generating content.

A Supervisor/Hierarchical Agent Team architecture is ideal because:
-   It allows for a central "SocialMediaManager" (supervisor) agent to interpret the user's intent and intelligently route the request to the most appropriate specialized worker agent.
-   Each worker agent (TrendAnalyst, SponsorFinder, ContentStrategist, ContentGenerator) can be highly focused and optimized for its specific task, potentially using different tools or prompt strategies.
-   This modularity enhances maintainability, scalability, and clarity of responsibilities within the graph.

Dry Runs of Use Cases:

1.  **'Identifying social media trends in sports.' (e.g., "trends in football")**
    *   **User Input:** "Tell me about trends in football."
    *   **SocialMediaManager:** Receives input, identifies intent as "identify trends," sets `next_agent` to "TrendAnalyst".
    *   **TrendAnalyst:** Receives the query, uses its LLM and potentially a `search_viral_content` tool (simulated) to find trends. Returns "Q&A reels are trending in football."
    *   **EndNode:** Formats the final response.
    *   **Output:** "Based on your query about trends in football, the Trend Analyst found that Q&A reels are currently trending."

2.  **'Finding relevant people for sponsoring specific posts.' (e.g., "people who would be interested in a post about UCL football")**
    *   **User Input:** "I need people interested in a post about UCL football for sponsorship."
    *   **SocialMediaManager:** Receives input, identifies intent as "find sponsors," sets `next_agent` to "SponsorFinder".
    *   **SponsorFinder:** Receives the query, uses its LLM and potentially an `influencer_search` tool (simulated) to identify relevant individuals. Returns "Current players like Dembele, Mbappe, and influencers focusing on European football."
    *   **EndNode:** Formats the final response.
    *   **Output:** "For your post about UCL football, the Sponsor Finder suggests current players like Dembele, Mbappe, and influencers focusing on European football."

3.  **'Suggesting content to post.' (e.g., "make a post about UCL football")**
    *   **User Input:** "I want to make a post about UCL football, what should I post?"
    *   **SocialMediaManager:** Receives input, identifies intent as "suggest content," sets `next_agent` to "ContentStrategist".
    *   **ContentStrategist:** Receives the query, uses its LLM to brainstorm content ideas, potentially considering trends (either from its own knowledge or by implicitly 'consulting' a trend-like function). Returns "A highlight reel of recent UCL matches would be highly engaging."
    *   **EndNode:** Formats the final response.
    *   **Output:** "Regarding your UCL football post, the Content Strategist suggests that a highlight reel of recent UCL matches would be highly engaging."

4.  **'Generating content based on a description.' (e.g., "Generate a script for a UCL highlight reel.")**
    *   **User Input:** "Generate a script for a UCL highlight reel focusing on top goals."
    *   **SocialMediaManager:** Receives input, identifies intent as "generate content," sets `next_agent` to "ContentGenerator".
    *   **ContentGenerator:** Receives the query, uses its LLM to generate a detailed script. Returns a multi-line script.
    *   **EndNode:** Formats the final response.
    *   **Output:** "Here is the generated content for your UCL highlight reel script: [Generated Script Content]"
"""

# --- Phase 2: Python Code Generation ---

# 1. State Definition
class GraphState(MessagesState):
    """
    Represents the state of our graph.

    Attributes:
        user_query: The initial query from the user.
        agent_output: The output generated by the last specialized agent.
        next_agent: The name of the next agent to route to, determined by the supervisor.
        final_response: The consolidated response to be presented to the user.
    """
    user_query: str = Field(default="")
    agent_output: str = Field(default="")
    next_agent: Literal["TrendAnalyst", "SponsorFinder", "ContentStrategist", "ContentGenerator", "END"] = Field(default="END")
    final_response: str = Field(default="")


# 2. Node Implementations (Python Functions)

# Initialize LLM
# TODO: Replace with your preferred LLM. Ensure API key is set as an environment variable.
# For OpenAI: OPENAI_API_KEY
# For Google: GOOGLE_API_KEY
llm = ChatOpenAI(model="gpt-4o", temperature=0) # Using gpt-4o for better reasoning and tool use
# llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

# Define tools (now using real implementations from snippet2)
@tool
def search_viral_content(query: str) -> str:
    """
    Searches for recent viral social media content or trending topics related to the query.
    Useful for identifying current trends in sports, entertainment, etc.
    Requires GOOGLE_API_KEY and GOOGLE_CSE_ID environment variables to be set.
    """
    # The GoogleSearchAPIWrapper will automatically pick up GOOGLE_API_KEY and GOOGLE_CSE_ID from os.environ
    search = GoogleSearchAPIWrapper()

    # Perform a search for the query, focusing on recent and trending aspects
    # Limiting to 5 results as a reasonable number for trending topics
    search_results = search.results(f"recent viral OR trending {query} social media", num_results=5)

    trending_content = []
    for result in search_results:
        # Combine title and snippet to form a descriptive string for each trending item
        content_string = f"Title: {result.get('title', 'N/A')}\nSnippet: {result.get('snippet', 'N/A')}"
        trending_content.append(content_string)
    
    # Join results into a single string, as the original tool returned a string
    return "\n\n".join(trending_content) if trending_content else "No trending content found."

@tool
def influencer_search(topic: str) -> str:
    """
    Searches for relevant influencers or public figures interested in a specific topic
    who might be suitable for sponsorships or collaborations.
    Requires GOLDEN_API_KEY environment variable to be set.
    """
    # The GoldenQueryAPIWrapper will automatically pick up GOLDEN_API_KEY from os.environ
    golden_query = GoldenQueryAPIWrapper()

    # Construct a query to find public figures or influencers related to the topic
    query = f"public figures interested in {topic}"

    try:
        response_str = golden_query.run(query)
        response_data = json.loads(response_str)

        influencers_info = []
        if 'results' in response_data:
            for item in response_data['results']:
                name = "Unknown"
                # Extract the name from the properties
                for prop in item.get('properties', []):
                    if prop.get('predicateId') == 'name' and prop.get('instances'):
                        name = prop['instances'][0]['value']
                        break
                
                # The Golden Query API, as per the provided documentation, does not
                # directly return 'platform' or 'relevance_score'.
                # These are placeholders to match the output schema if needed, but here we just format.
                influencers_info.append(f"Name: {name}, Platform: Unknown, Relevance: 0.0")
        
        # Join results into a single string, as the original tool returned a string
        return "Potential Influencers:\n" + "\n".join(influencers_info) if influencers_info else "No influencers found for this topic."
    except Exception as e:
        # Handle potential errors during API call or JSON parsing
        print(f"An error occurred during influencer search: {e}")
        return f"An error occurred during influencer search: {e}"


# Pydantic model for SocialMediaManager's output
class AgentDecision(BaseModel):
    """
    Represents the decision of the SocialMediaManager regarding the next agent to route to.
    """
    next_agent: Literal["TrendAnalyst", "SponsorFinder", "ContentStrategist", "ContentGenerator", "END"] = Field(
        description="The name of the specialized agent that should handle the user's request next, or 'END' if the task is complete or cannot be routed."
    )
    reasoning: str = Field(
        description="Brief explanation for the routing decision."
    )

# Bind tools to LLM for agents that might use them
llm_with_tools = llm.bind_tools([search_viral_content, influencer_search])

async def social_media_manager(state: GraphState) -> dict:
    """
    The central supervisor agent. It analyzes the user's query and determines which
    specialized agent should handle the request next.
    """
    print("---SOCIAL MEDIA MANAGER---")
    user_query = state.user_query
    messages = [
        SystemMessage(
            "You are a Social Media Manager. Your task is to analyze the user's request "
            "and determine which specialized agent should handle it. "
            "Choose from 'TrendAnalyst', 'SponsorFinder', 'ContentStrategist', 'ContentGenerator', or 'END' if the task is complete or cannot be routed."
            "The user's query is: " + user_query
        ),
        HumanMessage(content=user_query)
    ]

    # Use LLM with Pydantic output parser for structured decision making
    structured_llm = llm.with_structured_output(AgentDecision)
    decision: AgentDecision = await structured_llm.ainvoke(messages)

    print(f"Manager Decision: {decision.next_agent} (Reasoning: {decision.reasoning})")
    return {"next_agent": decision.next_agent, "messages": [HumanMessage(content=f"Manager decided to route to {decision.next_agent}.")]}

async def trend_analyst(state: GraphState) -> dict:
    """
    Specialized agent for identifying social media trends.
    """
    print("---TREND ANALYST---")
    user_query = state.user_query
    messages = [
        SystemMessage(
            "You are a Trend Analyst. Your goal is to identify social media trends "
            "based on the user's query. Use the 'search_viral_content' tool if relevant. "
            "Provide concise and actionable trend insights."
        ),
        HumanMessage(content=user_query)
    ]
    response = await llm_with_tools.ainvoke(messages)
    agent_output = response.content
    print(f"Trend Analyst Output: {agent_output}")
    return {"agent_output": agent_output, "messages": [HumanMessage(content=f"Trend Analyst completed task.")]}

async def sponsor_finder(state: GraphState) -> dict:
    """
    Specialized agent for finding relevant people or influencers for sponsoring specific posts.
    """
    print("---SPONSOR FINDER---")
    user_query = state.user_query
    messages = [
        SystemMessage(
            "You are a Sponsor Finder. Your task is to identify relevant people or influencers "
            "for sponsoring posts based on the user's description. Use the 'influencer_search' tool if relevant. "
            "List potential sponsors clearly."
        ),
        HumanMessage(content=user_query)
    ]
    response = await llm_with_tools.ainvoke(messages)
    agent_output = response.content
    print(f"Sponsor Finder Output: {agent_output}")
    return {"agent_output": agent_output, "messages": [HumanMessage(content=f"Sponsor Finder completed task.")]}

async def content_strategist(state: GraphState) -> dict:
    """
    Specialized agent for suggesting content ideas.
    """
    print("---CONTENT STRATEGIST---")
    user_query = state.user_query
    messages = [
        SystemMessage(
            "You are a Content Strategist. Your role is to suggest creative and engaging "
            "content ideas for social media based on the user's topic. Consider current trends "
            "and social media best practices. Be specific with your suggestions (e.g., 'a short video series', 'an interactive poll')."
        ),
        HumanMessage(content=user_query)
    ]
    response = await llm.ainvoke(messages) # No specific tools needed for this agent, just LLM reasoning
    agent_output = response.content
    print(f"Content Strategist Output: {agent_output}")
    return {"agent_output": agent_output, "messages": [HumanMessage(content=f"Content Strategist completed task.")]}

async def content_generator(state: GraphState) -> dict:
    """
    Specialized agent for generating actual content based on a description.
    """
    print("---CONTENT GENERATOR---")
    user_query = state.user_query
    messages = [
        SystemMessage(
            "You are a Content Generator. Your task is to generate specific content "
            "based on the user's detailed description. This could be text, a script, "
            "or a detailed outline. Be creative and fulfill the request precisely."
        ),
        HumanMessage(content=user_query)
    ]
    response = await llm.ainvoke(messages)
    agent_output = response.content
    print(f"Content Generator Output: {agent_output}")
    return {"agent_output": agent_output, "messages": [HumanMessage(content=f"Content Generator completed task.")]}

async def end_node(state: GraphState) -> dict:
    """
    A final node to consolidate the output from the specialized agents and prepare the final response.
    """
    print("---END NODE---")
    final_response = f"Based on your query: '{state.user_query}', here is the result from our social media assistant:\n\n{state.agent_output}"
    print(f"Final Response: {final_response}")
    return {"final_response": final_response, "messages": [HumanMessage(content=f"Final response generated.")]}

# 3. Graph Construction
graph_builder = StateGraph(GraphState)

# Add nodes
graph_builder.add_node("SocialMediaManager", social_media_manager)
graph_builder.add_node("TrendAnalyst", trend_analyst)
graph_builder.add_node("SponsorFinder", sponsor_finder)
graph_builder.add_node("ContentStrategist", content_strategist)
graph_builder.add_node("ContentGenerator", content_generator)
graph_builder.add_node("EndNode", end_node)

# Set entry point
graph_builder.add_edge(START, "SocialMediaManager")

# Define the routing function for the SocialMediaManager
def route_social_media_manager(state: GraphState) -> str:
    """
    Routes the graph based on the 'next_agent' field set by the SocialMediaManager.
    """
    print(f"Routing from SocialMediaManager to: {state.next_agent}")
    if state.next_agent == "TrendAnalyst":
        return "TrendAnalyst"
    elif state.next_agent == "SponsorFinder":
        return "SponsorFinder"
    elif state.next_agent == "ContentStrategist":
        return "ContentStrategist"
    elif state.next_agent == "ContentGenerator":
        return "ContentGenerator"
    else: # Default to END if no specific agent is identified or task is complete
        return END

# Add conditional edges from SocialMediaManager
graph_builder.add_conditional_edges(
    "SocialMediaManager",
    route_social_media_manager,
    {
        "TrendAnalyst": "TrendAnalyst",
        "SponsorFinder": "SponsorFinder",
        "ContentStrategist": "ContentStrategist",
        "ContentGenerator": "ContentGenerator",
        END: END # If the manager decides the task is complete or unroutable
    }
)

# Add regular edges from specialized agents to the EndNode
graph_builder.add_edge("TrendAnalyst", "EndNode")
graph_builder.add_edge("SponsorFinder", "EndNode")
graph_builder.add_edge("ContentStrategist", "EndNode")
graph_builder.add_edge("ContentGenerator", "EndNode")

# Add edge from EndNode to END
graph_builder.add_edge("EndNode", END)

# 4. Compilation
checkpointer = InMemoryCheckpointer()
final_app = graph_builder.compile(checkpointer=checkpointer)

# Example Usage (for testing)
async def run_graph(query: str):
    print(f"\n--- Running graph for query: '{query}' ---")
    inputs = {"user_query": query, "messages": [HumanMessage(content=query)]}
    async for s in final_app.astream(inputs):
        if "__end__" not in s:
            print(s)
            print("---")
    # After the stream, get the final state
    final_state = await final_app.aget_state(None)
    print(f"\nFinal State: {final_state.values}")
    print(f"\nFinal Response: {final_state.values['final_response']}")

if __name__ == "__main__":
    import asyncio

    # Test cases based on examples
    queries = [
        "Tell me about trends in football.",
        "I need people who would be interested in a post about UCL football for sponsorship.",
        "I want to make a post about UCL football, what should I post?",
        "Generate a script for a UCL highlight reel focusing on top goals.",
        "What is the weather like today?" # Example of a query that might lead to END
    ]

    for q in queries:
        asyncio.run(run_graph(q))
        print("\n" + "="*80 + "\n")

In [8]:
print(result["python_code"])

```python
import operator
import os
import json
from typing import Annotated, List, Literal, TypedDict

from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemoryCheckpointer
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import MessagesState

# New imports for the actual tool implementations
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_community.utilities.golden_query import GoldenQueryAPIWrapper

# --- Phase 1: Architectural Decision and Justification ---
"""
LangGraph Architecture: Agent Supervisor / Hierarchical Agent Teams

Justification:
The problem of "managing social media" naturally breaks down into distinct, specialized sub-tasks:
1.  Identifying social media trends.
2.  Finding relevant people for sponsoring posts.
3. 